### Predicted a list of ZTF objects.

In [1]:
from generate_annotator import *
import pandas as pd 
import lasair, os
import settings
import numpy as np 


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [7]:


def predict_targeted_ZTF(objectId, L,  threshold = 0.75):

    objectInfo = L.objects([objectId])[0]
    classification = None
    probs = None
    # print(objectInfo)
    if objectInfo is None:
        print('objectInfo is None.')
        return classification, probs
    else:
        img_data, meta_data = collect_data_from_lasair(objectId, objectInfo, band = 'r')

        if img_data is None or meta_data is None:
            print('image data is None or meta data is None.')
            return classification, probs
        else:
            results = needle_prediction(img_data, meta_data)
            probs = 'SN: %f, SLSN-I: %f, TDE: %f' % (results[0][0], results[0][1], results[0][2])
            
            if np.max(results[0]) >= threshold:
                classification = LABEL_LIST[np.argmax(results[0])]
            else:
                classification = 'unclear'
            return classification, probs
    

In [11]:
objectId = 'ZTF24aannvty'
L = lasair.lasair_client(settings.API_TOKEN)
predict_targeted_ZTF(objectId, L,  threshold = 0.75)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20160  100 20160    0     0   147k      0 --:--:-- --:--:-- --:--:--  152k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

1/1 [==============================] - 0s 15ms/step


100 20160  100 20160    0     0   126k      0 --:--:-- --:--:-- --:--:--  128k


1/1 [==============================] - 0s 13ms/step
https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr2/stack.json?ra=191.040938318&dec=-4.23272662&radius=0.00139&columns=[raStack,decStack,gPSFMag,gPSFMagErr,rPSFMag,rPSFMagErr,iPSFMag,iPSFMagErr,zPSFMag,zPSFMagErr,yPSFMag,yPSFMagErr, gApMag,gApMagErr,rApMag,rApMagErr,iApMag,iApMagErr,zApMag,zApMagErr,yApMag,yApMagErr,yKronMag]&nDetections.gte=6&pagesize=10000
INFO: Loading the maps from /Users/xinyuesheng/.extinction/maps//maps.yaml
 -  sfd is loaded
Success! File created: ZTF24aannvty.csv
meta data:  [19.8203125, 20.59589958190918, -0.77559, 43.0149, -0.01413873136410408, -0.018030728886967076, -1.97981, 22.033954639113137, 21.80012551486469, 21.373722364927183, 21.210570284712812, 21.285431281095654, 0.2338291242484445, 0.4264031499375029, 0.386]
test:  /Users/xinyuesheng/Documents/astro_projects/scripts/classifier_v1/model_with_data/r_band/lasair_20240105/
1/1 [==============================] - 0s 91ms/step


('SLSN-I', 'SN: 0.019797, SLSN-I: 0.962638, TDE: 0.017566')

true positive [  0.12559969 -19.38178567  22.89091652  -0.15900035 -11.49703436
   0.2205368    0.37508952  -0.20549637  -0.31197533  -0.27176676
  -0.34038387  -0.29961277   0.35463699  -0.16289772  -0.5776914 ]

false negative [ 1.59156885  0.98102282  0.44909883  0.84735426  0.03654531  0.03800414
  0.56019562  0.18701283  0.19120783  0.17812879  0.16981192  0.17840142
 -0.03110243  0.04488724 -0.58028887]

In [ ]:
# write a csv file for storing the prediction results


data = pd.read_csv('20220301_20240225.csv')
obj_list = data['object_id'].to_list()
obj_labels = data['type'].to_list()
L = lasair.lasair_client(settings.API_TOKEN)
results = []
for obj, label in zip(obj_list, obj_labels):
    classification, probs = predict_targeted_ZTF(obj, L, 0.75)
    results.append([obj, label, classification, probs])





In [ ]:

df = pd.DataFrame(results, columns = ['ztf_id', 'type', 'classification', 'probability'])
df.to_csv('20220301_20240225_needle.csv')

In [ ]:
df

,ztf_id,type,classification,probability
0,ZTF23abofwba,SLSN-I,None,None
1,ZTF23abjarpv,SLSN-I,None,None
2,ZTF23aboebgh,SLSN-I,None,None
3,ZTF23aaznlgb,SLSN-I,None,None
4,ZTF23abawhql,SLSN-I,None,None
5,ZTF23aawhcjb,SLSN-I,None,None
6,ZTF23aagdbbv,SLSN-I,None,None
7,ZTF23aaarvxj,SLSN-I,None,None
8,ZTF22abynkpz,SLSN-I,None,None
9,ZTF22abvarjh,SLSN-I,None,None
